In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [63]:
def get_coef(MOB):
    Z0 = MOB[:, -1][:-2]
    A0 = Z0.sum()
    AJ = MOB.sum(0)[:-1]
    
    a = MOB[:-2, :-1] / np.tile(AJ, (MOB.shape[0] - 2, 1))
    b = MOB[-2:, :-1] / np.tile(AJ, (2, 1))
    a0 = Z0 / A0
    
    X = MOB[:-2, :-1]
    l = MOB[-2:, :-1]
    return Z0, A0, AJ, a, b, a0, X, l

def get_alpha(Z, l, a, b, AJ):
    return AJ * (1 / Z ** a).prod(0) * (1 / l ** b).prod(0)

def get_FJ(X, l, a, b, alpha):
    return (alpha * (X ** a).prod(0) * (l ** b).prod(0)).astype(float)

def get_F0(A0, X0, Z0, a0):
    return A0 * (1 / Z0 ** a0).prod() * (X0 ** a0).prod()

def get_GDP(X, FJ, F0, a, A0):
    d = np.log(FJ.astype(float))
    mu = -np.linalg.inv(np.eye(X.shape[0]) - X.T) @ d
    lam = np.exp((mu * a).sum()) / F0
    return A0 / lam

def add_shock_to_production(MOB, a, shock_mask):
    Y = MOB.sum(0)[:-1]
    X0 = (np.eye(a.shape[0]) - a) @ (Y * shock_mask)
    X = a * np.tile(Y, (a.shape[0], 1))
    return X.astype(float), X0.astype(float)

def add_shock_to_demand(MOB, a, shock_mask):
    Z = MOB[:-2, -1]
    Y = np.linalg.inv(np.eye(a.shape[0]) - a) @ (Z * shock_mask)
    X = a * np.tile(Y, (a.shape[0], 1))
    return X.astype(float), Y.astype(float)

def get_GDP_from_df(df, shock):
    MOB = np.array(df)[:, :]
    Z0, A0, AJ, a, b, a0, Z, l = get_coef(MOB)
    alpha = get_alpha(Z, l, a, b, AJ)
    FJ = lambda X: get_FJ(X, l, a, b, alpha*shock)
    F0 = lambda X0: get_F0(A0, X0, Z0, a0)
    X, X0 = add_shock_to_production(MOB, a, shock)
    return get_GDP(X, FJ(X), F0(X0), a, A0)

In [177]:
df_2019

,Unnamed: 0,"Agriculture, forestry",Mining,Processing industries,Electric power supply and other,"Water supply, sanitation and other",Construction,Wholesale and retail trade,Transportation and storage,Operation of hotels and other,Information and communication,Operations with real estate,Education,Healthcare,Other activities,Usage
0,"Agriculture, forestry",11888.465358,0.545273,29979.669725,39.646382,0.638771,159.990823,90.077346,79.239532,519.480449,0.606832,4.567212,84.190007,89.651498,498.965257,80522.941622
1,Mining,13.059125,1337.881465,22099.810250,3390.941404,10.757061,668.717148,1035.698555,487.787924,0.457338,0.269023,4.715582,2.144494,4.465775,58.078135,77006.542924
2,Processing industries,13535.526810,5238.444446,143085.511116,5361.492925,3115.667714,39345.819329,7769.738386,22050.292195,4118.887885,2540.377940,3929.239474,1325.985354,4270.599498,13265.889746,670262.104102
3,Electric power supply and other,1414.101993,2310.559133,15384.678297,26865.183272,873.197212,792.542032,2035.184463,4234.198808,433.246016,573.800797,3781.111556,1651.099603,1029.976495,3239.383867,71943.577377
4,"Water supply, sanitation and other",57.693202,84.327668,4392.573859,594.916872,1371.225610,158.675982,319.329512,159.581128,64.530988,47.057740,859.849093,182.711537,171.721887,902.754968,11188.574504
5,Construction,206.681092,1693.523369,2091.045017,1071.317591,245.618619,2515.600187,545.134540,1720.859270,155.612780,112.921576,3621.061879,1495.400640,1034.467474,5745.496307,154992.390101
6,Wholesale and retail trade,3935.999295,1163.031927,40393.325550,10018.689437,1095.317473,10428.995934,9229.382266,6033.549221,1270.514955,863.251156,1955.189350,492.055448,2765.947232,5031.532181,369555.243782
7,Transportation and storage,2046.023385,6718.795674,36117.174055,1038.813149,633.320774,4464.445401,41073.660567,37186.229885,290.376111,674.970002,309.515147,262.292094,577.636252,9368.309638,174374.069975
8,Operation of hotels and other,12.510688,49.566509,333.470446,59.674684,5.292151,183.720971,241.469269,229.499701,44.908293,94.923223,14.004179,355.115235,450.882605,1683.780625,29594.873305
9,Information and communication,110.825889,172.863421,3450.608090,789.787141,62.952015,541.624654,3935.149186,1650.529380,154.909578,22528.024648,708.896716,884.140449,503.657732,14273.153821,76962.651865


In [198]:
# idx_region = 76 # novosib
# str_region = 'novosib'

# idx_region = 77 # omsk
# str_region = 'omsk'

# idx_region = 10 # lipezk
# str_region = 'lipezk'

# idx_region = 27 # murmansk
# str_region = 'murmansk'

idx_region = 38 # rostov
str_region = 'rostov'
df_shock = pd.read_excel('../data/data_for_analysis/df_shocks_2019.xlsx')
df_2019 = pd.read_excel('../data/data_for_analysis/df_'+str_region+'_2019.xlsx').set_index('Unnamed: 0')
df_2020 = pd.read_excel('../data/data_for_analysis/df_'+str_region+'_2020.xlsx').set_index('Unnamed: 0')

shock = np.ones(14)
shock_vvp = df_shock['vvp_'+str(idx_region)]
shock_labour = df_shock['labour_'+str(idx_region)]
res_2019 = get_GDP_from_df(df_2019, shock)
res_2019_vvp = get_GDP_from_df(df_2019, shock_vvp)
res_2019_labour = get_GDP_from_df(df_2019, shock_labour)
res_2020 = get_GDP_from_df(df_2020, shock)

In [199]:
res_2020 / res_2019 * 100

95.30846040334262

In [200]:
res_2019_vvp / res_2019

1.0008772001631776

In [201]:
res_2019_labour / res_2019

0.9939731424806715